In [12]:
# Importation des bibliothèques nécessaires
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
import json
from typing import List, Optional

DATA_PATH = "..\\data\\"


Import de la dernière base d'oeufs stockée dans data (extraite du fichier parquet) avec résultat de l'analyse : breeding, quantity, french +résultat de l'OCR

In [13]:
df_oeufs_fr = pd.read_csv("../data/eggs_is_suffering_computed_fr.csv")
eggs = pd.read_csv(DATA_PATH+"eggs_is_suffering_computed.csv")
df_oeufs_fr

,code,categories_tags,labels_tags,product_name,generic_name,quantity,product_quantity_unit,product_quantity,allergens_tags,ingredients_tags,...,weight_from_OCR,quantity_from_OCR,size_from_OCR,w_eggs,breeding,has_breeding_type,has_egg_weight,has_egg_weight_s,french,french_s
0,00003100,"['en:farming-products', 'en:eggs']",[],"[{'lang': 'main', 'text': 'Hard Boiled Eggs'},...",[],2,NaN,0.0,['en:eggs'],"['fr:eggs', 'en:e330', 'fr:sodium-benzoate', '...",...,100.0,2.0,NaN,100.0,unknown,unknown,True,has weight,True,français
1,00257961,"['en:farming-products', 'en:eggs']",NaN,"[{'lang': 'main', 'text': '6 Oeufs Bleus'}, {'...",[],NaN,NaN,NaN,[],NaN,...,0.0,0.0,NaN,0.0,unknown,unknown,False,no weight,True,français
2,0055000002240,"['en:farming-products', 'en:eggs']",[],"[{'lang': 'main', 'text': 'Œufs'}, {'lang': 'f...",[],NaN,NaN,NaN,[],NaN,...,0.0,0.0,NaN,0.0,unknown,unknown,False,no weight,True,français
3,0061719011930,"['en:farming-products', 'en:eggs', 'en:chicken...",[],"[{'lang': 'main', 'text': '12 œufs blanc calib...",[],12,NaN,0.0,[],NaN,...,720.0,14.4,NaN,600.0,barn,computed,True,has weight,True,français
4,00901796,"['en:farming-products', 'en:eggs', 'en:chicken...","['en:green-dot', 'fr:categorie-a']","[{'lang': 'main', 'text': '6 Œufs Moyens de Po...",[],6 œufs,NaN,0.0,['en:eggs'],['en:egg'],...,300.0,6.0,NaN,300.0,free_range,free_range,True,has weight,True,français
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2612,0764450111661,"['en:farming-products', 'en:eggs']","['en:nutriscore', 'fr:bleu-blanc-coeur']",[],[],NaN,NaN,NaN,[],NaN,...,0.0,0.0,NaN,0.0,unknown,unknown,False,no weight,True,français
2613,3700864040064,"['en:farming-products', 'en:eggs', 'en:chicken...",['en:nutriscore'],"[{'lang': 'main', 'text': 'Gros oeufs plein ai...",[],NaN,NaN,NaN,[],NaN,...,0.0,0.0,NaN,0.0,free_range,free_range,False,no weight,True,français
2614,3700133918872,"['en:farming-products', 'en:eggs', 'en:chicken...",['fr:label-rouge'],"[{'lang': 'main', 'text': 'Oeufs frais plein a...",[],NaN,NaN,NaN,[],NaN,...,0.0,0.0,NaN,0.0,free_range,free_range,False,no weight,True,français
2615,3700864017127,"['en:farming-products', 'en:eggs', 'en:chicken...",[],"[{'lang': 'main', 'text': 'Gros oeufs plein ai...",[],NaN,NaN,NaN,[],NaN,...,0.0,0.0,NaN,0.0,free_range,free_range,False,no weight,True,français


Définition des listes des codes produits à recenser sur l'excel :

- ceux dont élevage et quantité manquent
- ceux dont élevage manque
- ceux qui ne sont pas plein air et dont quantité manque


In [14]:
# Filter products with no breeding type and no quantity
dfoeufs_fr_no_breeding_no_quantity = df_oeufs_fr.loc[
    (df_oeufs_fr["has_breeding_type"].isin(["Aucun", "unknown"])) &
    (df_oeufs_fr["has_egg_weight"] == False), 
    'code'
]

# Filter products with no breeding type but with quantity
dfoeufs_fr_no_breeding = df_oeufs_fr.loc[
    (df_oeufs_fr["has_breeding_type"].isin(["Aucun", "unknown"])) &
    (df_oeufs_fr["has_egg_weight"] == True), 
    'code'
]

# Filter products with breeding type (not free range) but no quantity
dfoeufs_fr_no_quantity_not_free_range = df_oeufs_fr.loc[
    (df_oeufs_fr["has_breeding_type"] != "free_range") &
    (df_oeufs_fr["has_breeding_type"] != "Aucun") & 
    (df_oeufs_fr["has_breeding_type"] != "unknown") & 
    (df_oeufs_fr["has_egg_weight"] == False), 
    'code'
]

# Filter products with breeding type but no quantity (non-utilisé)
dfoeufs_fr_no_quantity = df_oeufs_fr.loc[
    (df_oeufs_fr["has_breeding_type"] != "Aucun") & 
    (df_oeufs_fr["has_breeding_type"] != "unknown") & 
    (df_oeufs_fr["has_egg_weight"] == False), 
    'code'
]

# Convert to lists
no_breeding_no_quantity = dfoeufs_fr_no_breeding_no_quantity.tolist()
no_breeding = dfoeufs_fr_no_breeding.tolist()
no_quantity = dfoeufs_fr_no_quantity.tolist()
no_quantity_not_free_range = dfoeufs_fr_no_quantity_not_free_range.tolist()

In [15]:
import json
import os
import pandas as pd
import numpy as np
from typing import List, Optional, Dict, Any
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.utils import get_column_letter


class ExcelProductGenerator:
    """Compact Excel generator for product data from DataFrame."""
    
    # Configuration constants
    CONFIG = {
        'sheet_name': "Feuille1",
        'excel_ext': ".xlsx",
        'image_count': 10,
        'row_height': 300,
        'freeze_offset': 2
    }
    
    # Column definitions
    COLUMNS = {
        'base': ["Traité", "Info invisible", "Pas œuf", "Code et URL produit", "Autres"],
        'optional': {"breeding": "Elevage", "quantity": ["Qté", "Calibre"]},
        'new': ["OCR", "weight from OCR", "breeding type from OCR", "breeding_type_related", "weight_type_related", "predictions_categories"],
        'images': [f"Image {i}" for i in range(1, 11)]
    }
    
    # Column order definition (matches original COLUMN_ORDER)
    COLUMN_ORDER = [
        "Traité",
        "Info invisible", 
        "Elevage",  # Optional
        "Qté", "Calibre",  # Optional quantity columns
        "Pas œuf",
        "Code et URL produit",
        "Autres",
        "Image 1",
        "weight from OCR",
        "breeding type from OCR",
        "OCR",
        "breeding_type_related", 
        "weight_type_related",
        "predictions_categories"
    ] + [f"Image {i}" for i in range(2, 11)]
    

    
    # Renaming and formatting
    RENAMES = {
        "Traité": "Traité",
        "Pas œuf": "Pas œuf / comment.",
        "Autres": "Nom, ingrédients, labels, catégories",
        "Elevage": "Elevage\n (cage, sol, plein-air, bio ou 3, 2, 1, 0)",
        "Qté": "Quantité\n(nb d'œufs ou poids)",
        "Calibre": "Calibre\n(S, M, L, XL ou petit, moyen, gros, très gros)",
        "predictions_categories": "predictions categories"
    }
    
    WIDTHS = {
        "Traité": 8, "Info invisible": 10, "Pas œuf / comment.": 10,
        "Code et URL produit": 15, "Nom, ingrédients, labels, catégories": 22,
        "Image principale": 50, "Image ingrédients": 50,
        "Elevage\n (cage, sol, plein-air, bio ou 3, 2, 1, 0)": 15,
        "Quantité\n(nb d'œufs ou poids)": 13,
        "Calibre\n(S, M, L, XL ou petit, moyen, gros, très gros)": 17,
        "weight from OCR": 20, "breeding type from OCR": 20,
        "OCR": 50, "breeding_type_related": 15, "weight_type_related": 15,
        "predictions categories": 15,
        **{f"Image {i}": 50 for i in range(1, 11)}
    }

    def __init__(self, data_path: str = DATA_PATH):
        self.data_path = data_path

    def _parse_json_field(self, field: Any) -> Any:
        """Parse JSON-like string fields."""
        if not isinstance(field, str):
            return field
        try:
            return json.loads(field.replace("'", '"').replace("None", "null"))
        except json.JSONDecodeError:
            return field

    def _extract_text_list(self, data: Any) -> str:
        """Extract text from list of dictionaries."""
        parsed = self._parse_json_field(data)
        if isinstance(parsed, list):
            return '\n'.join(
                d.get('text', '') for d in parsed 
                if isinstance(d, dict) and d.get('lang') != "main"
            )
        return str(parsed) if parsed else ""

    def _get_df_value(self, code: str, df: pd.DataFrame, column: str) -> Optional[str]:
        """Get value from DataFrame for specific code and column."""
        try:
            # Convertir le code en string et s'assurer que la colonne code du DataFrame est aussi en string
            code_str = str(code)
            df_codes = df['code'].astype(str)
            mask = df_codes == code_str
            if mask.any():
                value = df.loc[mask, column].iloc[0]
                return str(value) if pd.notna(value) else None
        except (KeyError, IndexError) as e:
            print(f"Warning: Column '{column}' not found or error accessing data for code {code}: {e}")
        return None

    def _build_image_urls(self, code: str, images: Any) -> List[str]:
        """Build image URLs from product images data."""
        try:
            images_list = self._parse_json_field(images) if images else []
            if not isinstance(images_list, list):
                return []
                
            # Get sorted image keys
            image_keys = sorted([
                img["key"] for img in images_list
                if isinstance(img, dict) and str(img.get("key")).isdigit()
            ], key=int)
            
            # Build URLs
            code_padded = str(code).zfill(13)
            path = f"{code_padded[:3]}/{code_padded[3:6]}/{code_padded[6:9]}/{code_padded[9:]}"
            
            return [
                f"https://images.openfoodfacts.net/images/products/{path}/{k}.jpg"
                for k in image_keys
            ][:self.CONFIG['image_count']]
        except Exception as e:
            print(f"Warning: Error building image URLs for code {code}: {e}")
            return []

    def _get_new_columns_data(self, code: str, df: pd.DataFrame) -> Dict[str, str]:
        """Get data for new columns."""
        # Get probability values and join non-empty ones
        probas = [self._get_df_value(code, df, f'proba_{i}') or "" for i in range(1, 4)]
        proba_text = "\n".join(p for p in probas if p)
        
        # Get weight from OCR data
        weight_ocr_fields = ['quantity_from_OCR', 'size_from_OCR', 'weight_from_OCR']
        weight_ocr_values = [self._get_df_value(code, df, field) or "" for field in weight_ocr_fields]
        weight_ocr_text = "\n".join(v for v in weight_ocr_values if v)
        
        # Get breeding type from OCR data
        breeding_ocr_fields = ['cage_from_OCR_2', 'barn_from_OCR_2', 'free_range_from_OCR_2', 'organic_from_OCR_2']
        breeding_ocr_values = [self._get_df_value(code, df, field) or "" for field in breeding_ocr_fields]
        breeding_ocr_text = "\n".join(v for v in breeding_ocr_values if v)
        
        return {
            'OCR': self._get_df_value(code, df, 'texte_ocr') or "",
            'weight from OCR': weight_ocr_text,
            'breeding type from OCR': breeding_ocr_text,
            'breeding_type_related': self._get_df_value(code, df, 'breeding_type_related') or "",
            'weight_type_related': self._get_df_value(code, df, 'weight_type_related') or "",
            'predictions_categories': proba_text
        }

    def build_row(self, product: dict, code: str, df: pd.DataFrame, 
                  show_breeding: bool, show_quantity: bool,
                  breeding_value: Optional[str] = None,
                  quantity_value: Optional[str] = None) -> dict:
        """Build complete row from product data."""
        
        try:
            # Extract and format product info
            product_name = self._extract_text_list(product.get("product_name", ""))
            generic_name = self._extract_text_list(product.get("generic_name", ""))
            
            # Combine product details
            details = f"{product_name}\n{generic_name}\n" + \
                     f"{product.get('ingredients_tags', [])}\n" + \
                     f"{product.get('labels_tags', [])}\n" + \
                     f"{product.get('categories_tags', [])}"
            
            # Build image URLs and create image columns
            image_urls = self._build_image_urls(code, product.get("images", []))
            image_cols = {
                col: f'=IMAGE("{image_urls[i]}")' if i < len(image_urls) else ""
                for i, col in enumerate(self.COLUMNS['images'])
            }
            
            # Base row structure
            row = {
                "Traité": "", "Info invisible": "", "Pas œuf": "",
                "Code et URL produit": str(code), "Autres": details,
                **image_cols
            }
            
            # Add optional columns
            if show_breeding:
                row["Elevage"] = breeding_value or ""
            if show_quantity:
                row.update({"Qté": quantity_value or "", "Calibre": ""})
            
            # Add new columns
            row.update(self._get_new_columns_data(code, df))
            
            return row
            
        except Exception as e:
            print(f"Warning: Error building row for code {code}: {e}")
            # Retourner une ligne basique même en cas d'erreur
            return {
                "Traité": "", "Info invisible": "", "Pas œuf": "",
                "Code et URL produit": str(code), "Autres": f"Erreur: {str(e)}",
                **{col: "" for col in self.COLUMNS['images']},
                **({'Elevage': ""} if show_breeding else {}),
                **({'Qté': "", 'Calibre': ""} if show_quantity else {}),
                **{col: "" for col in self.COLUMNS['new']}
            }

    def _create_column_order(self, show_breeding: bool, show_quantity: bool) -> List[str]:
        """Create column order based on options, filtering from COLUMN_ORDER."""
        columns = []
        
        for col in self.COLUMN_ORDER:
            # Always include base columns
            if col in ["Traité", "Info invisible", "Pas œuf", "Code et URL produit", "Autres"]:
                columns.append(col)
            # Include breeding if requested
            elif col == "Elevage" and show_breeding:
                columns.append(col)
            # Include quantity columns if requested  
            elif col in ["Qté", "Calibre"] and show_quantity:
                columns.append(col)
            # Always include new columns and images
            elif col in self.COLUMNS['new'] or col in self.COLUMNS['images']:
                columns.append(col)
        
        return columns

    def _apply_formatting(self, filename: str, columns: List[str], codes: List[str], sheet_name: str):
        """Apply Excel formatting."""
        wb = load_workbook(filename)
        ws = wb[sheet_name]

        # Hyperlinks for product codes
        code_col = columns.index("Code et URL produit") + 1
        for i, code in enumerate(codes, 2):
            cell = ws.cell(row=i, column=code_col)
            cell.hyperlink = f"https://fr.openfoodfacts.org/produit/{code}"
            cell.style = "Hyperlink"

        # Column widths and row heights
        for idx, name in enumerate(columns, 1):
            ws.column_dimensions[get_column_letter(idx)].width = self.WIDTHS.get(name, 15)
        
        for r in range(2, len(codes) + 2):
            ws.row_dimensions[r].height = self.CONFIG['row_height']

        # Styles
        fill_gray = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
        border = Border(
            left=Side(style="dotted", color="E0E0E0"),
            right=Side(style="dotted", color="E0E0E0"),
            top=Side(style="thin", color="AAAAAA"),
            bottom=Side(style="thin", color="AAAAAA")
        )

        # Apply formatting to all cells
        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.alignment = Alignment(wrap_text=True, vertical="center", horizontal="center")
                cell.border = border

        # Header formatting
        for cell in ws[1]:
            cell.font = Font(size=8, bold=True)

        # Freeze panes and gray fill
        if "Pas œuf / comment." in columns:
            freeze_col = columns.index("Pas œuf / comment.") + self.CONFIG['freeze_offset']
            ws.freeze_panes = ws.cell(row=2, column=freeze_col)
            
            for r in range(2, ws.max_row + 1):
                for c in range(freeze_col, ws.max_column + 1):
                    ws.cell(row=r, column=c).fill = fill_gray

        wb.save(filename)
        print(f"✅ Excel sheet '{sheet_name}' generated in {filename}")

    def generate_excel(self, product_codes: List[str], df: pd.DataFrame, 
                      output_filename: str, show_col_breeding: bool = True,
                      show_cols_quantity: bool = True, fetch_breeding: bool = False,
                      fetch_quantity: bool = False, 
                      sheet_name: str = None) -> str:
        """Generate Excel file from product codes and DataFrame."""
        
        sheet_name = sheet_name or self.CONFIG['sheet_name']
        
        # Prepare file path
        if not output_filename.endswith(self.CONFIG['excel_ext']):
            output_filename += self.CONFIG['excel_ext']
        file_path = os.path.join(self.data_path, output_filename) if self.data_path else output_filename
        
        # Convertir les codes en string pour assurer la cohérence
        product_codes_str = [str(code) for code in product_codes]
        df_codes_str = df['code'].astype(str) if 'code' in df.columns else pd.Series()
        
        print(f"🔍 Processing {len(product_codes_str)} product codes...")
        print(f"📊 DataFrame contains {len(df)} rows with codes column: {'code' in df.columns}")
        
        # Process products - TOUJOURS inclure tous les codes, même si pas dans le DataFrame
        rows, valid_codes, missing_codes = [], [], []
        
        for code in product_codes_str:
            try:
                # Chercher le produit dans le DataFrame
                if 'code' in df.columns and code in df_codes_str.values:
                    product_row = df[df_codes_str == code]
                    if not product_row.empty:
                        product = product_row.iloc[0].replace({np.nan: None}).to_dict()
                        
                        # Get optional values
                        breeding_val = self._get_df_value(code, df, 'breeding') if fetch_breeding else None
                        quantity_val = self._get_df_value(code, df, 'w_eggs') if fetch_quantity else None
                        
                        row = self.build_row(product, code, df, show_col_breeding, 
                                           show_cols_quantity, breeding_val, quantity_val)
                        valid_codes.append(code)
                    else:
                        # Produit pas trouvé dans le DataFrame - créer ligne vide
                        row = self._create_empty_row(code, show_col_breeding, show_cols_quantity)
                        missing_codes.append(code)
                else:
                    # DataFrame n'a pas de colonne 'code' ou code pas trouvé - créer ligne vide
                    row = self._create_empty_row(code, show_col_breeding, show_cols_quantity)
                    missing_codes.append(code)
                
                rows.append(row)
                
            except Exception as e:
                print(f"⚠️  Error processing product {code}: {e}")
                # Même en cas d'erreur, on inclut le code avec une ligne d'erreur
                row = self._create_error_row(code, str(e), show_col_breeding, show_cols_quantity)
                rows.append(row)
                missing_codes.append(code)
        
        # Statistiques
        print(f"✅ Successfully processed: {len(valid_codes)} products")
        print(f"⚠️  Missing from DataFrame: {len(missing_codes)} products")
        print(f"📝 Total rows generated: {len(rows)}")
        
        if missing_codes:
            print(f"Missing codes: {missing_codes[:10]}{'...' if len(missing_codes) > 10 else ''}")
        
        if not rows:
            raise ValueError("No products could be processed at all")
        
        # Create DataFrame
        columns = self._create_column_order(show_col_breeding, show_cols_quantity)
        df_output = pd.DataFrame(rows).reindex(columns=columns)
        df_output.rename(columns=self.RENAMES, inplace=True)
        
        # Write to Excel
        if os.path.exists(file_path):
            wb = load_workbook(file_path)
            if sheet_name in wb.sheetnames:
                del wb[sheet_name]
            ws = wb.create_sheet(title=sheet_name)
        else:
            wb = Workbook()
            ws = wb.active
            ws.title = sheet_name

        # Write data
        for col_idx, col_name in enumerate(df_output.columns, 1):
            ws.cell(row=1, column=col_idx, value=col_name)
        
        for row_idx, row_data in enumerate(df_output.itertuples(index=False), 2):
            for col_idx, value in enumerate(row_data, 1):
                ws.cell(row=row_idx, column=col_idx, value=value)
        
        wb.save(file_path)
        
        # Apply formatting - utiliser tous les codes traités
        all_processed_codes = valid_codes + missing_codes
        self._apply_formatting(file_path, df_output.columns.tolist(), all_processed_codes, sheet_name)
        
        return file_path

    def _create_empty_row(self, code: str, show_breeding: bool, show_quantity: bool) -> dict:
        """Create empty row for missing product."""
        row = {
            "Traité": "", "Info invisible": "", "Pas œuf": "",
            "Code et URL produit": str(code), "Autres": "Produit non trouvé dans les données",
            **{col: "" for col in self.COLUMNS['images']},
            **{col: "" for col in self.COLUMNS['new']}
        }
        
        if show_breeding:
            row["Elevage"] = ""
        if show_quantity:
            row.update({"Qté": "", "Calibre": ""})
            
        return row
        
    def _create_error_row(self, code: str, error: str, show_breeding: bool, show_quantity: bool) -> dict:
        """Create error row for failed product processing."""
        row = {
            "Traité": "", "Info invisible": "", "Pas œuf": "",
            "Code et URL produit": str(code), "Autres": f"Erreur: {error}",
            **{col: "" for col in self.COLUMNS['images']},
            **{col: "" for col in self.COLUMNS['new']}
        }
        
        if show_breeding:
            row["Elevage"] = ""
        if show_quantity:
            row.update({"Qté": "", "Calibre": ""})
            
        return row


# Backward compatibility wrapper
def generate_product_excel_from_df(product_codes: List[str], df: pd.DataFrame,
                                   output_excel_filename: str, show_col_breeding: bool = True,
                                   show_cols_quantity: bool = True, fetch_breeding: bool = True,
                                   fetch_quantity: bool = True, sheet_name: str = "Feuille1") -> str:
    """Backward compatibility wrapper."""
    generator = ExcelProductGenerator()
    return generator.generate_excel(
        product_codes, df, output_excel_filename, show_col_breeding,
        show_cols_quantity, fetch_breeding, fetch_quantity, sheet_name
    )

In [16]:
# Afficher les produits contenant des ingredients de type "barn"
confirm = input("Créer l'excel de test ? (o/n): ")
if confirm.lower() == 'o':
    print("Création d'un excel de test pour un échantillon de 20 produits")
    sample_codes = eggs.sample(20)['code'].tolist()
    print(sample_codes)

    products_test = generate_product_excel_from_df(df = eggs,
        product_codes=sample_codes,
        output_excel_filename="products_test.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        sheet_name="Test produits",
        fetch_breeding=True,
        fetch_quantity=True,
    )
    os.startfile(products_test)


else:
    print("Opération passée")

Création d'un excel de test pour un échantillon de 20 produits
['3596710479498', '0746025606086', '3503015568062', '8411384100040', '8011752105266', '0026855123824', '8001300643586', '3701002407817', '3282620030422', '4061459735048', '6410402016242', '8033520000610', '8711427003704', '5010482666705', '7501194312014', '8426510014719', '5060343740075', '6191314600018', '2004060087238', '3760210731060']
🔍 Processing 20 product codes...
📊 DataFrame contains 7650 rows with codes column: True
✅ Successfully processed: 20 products
⚠️  Missing from DataFrame: 0 products
📝 Total rows generated: 20
✅ Excel sheet 'Test produits' generated in ..\data\products_test.xlsx


In [17]:
# Afficher les produits contenant des ingredients de type "barn"
all_codes = eggs['code'].tolist()

confirm = input(f"Créer l'excel pour tous les produits monde ({len(all_codes)} produits - 3 minutes) ?")

if confirm.lower() == 'o':
    print("Création d'un excel de test pour tous les produits monde")
    all_products = generate_product_excel_from_df(df = eggs,
        product_codes=all_codes,
        output_excel_filename="all_products.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        sheet_name="all products",
        fetch_breeding=True,
        fetch_quantity=True,
    )
    os.startfile(all_products)

else:
    print("Opération passée")

Opération passée


In [18]:
# Afficher les produits contenant des ingredients de type "barn"
confirm = input(f"Créer l'excel pour tous les produits fr  avec info manquante \
                (({len(no_breeding_no_quantity)+ len(no_breeding) + len(no_quantity_not_free_range)} produits - 10 secondes)? (o/n): ")

if confirm.lower() == 'o':
    generate_product_excel_from_df(df = df_oeufs_fr,
        product_codes=no_breeding_no_quantity,
        output_excel_filename="products_to_tag_14_06_25.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        fetch_breeding=False,
        fetch_quantity=False,
        sheet_name="no_breeding_no_quantity"
    )

    generate_product_excel_from_df(df = df_oeufs_fr,
        product_codes=no_breeding,
        output_excel_filename="products_to_tag_14_06_25.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        sheet_name="no_breeding",
        fetch_breeding=False,
        fetch_quantity=True,
    )

    generate_product_excel_from_df(df = df_oeufs_fr,
        product_codes=no_quantity_not_free_range,
        output_excel_filename="products_to_tag_14_06_25.xlsx",
        show_col_breeding=True,
        show_cols_quantity=True,
        fetch_breeding=True,
        fetch_quantity=False,
        sheet_name="no_quantity_not_free_range"
    )
    os.startfile(os.path.join(DATA_PATH, "products_to_tag_14_06_25.xlsx"))

else:
    print("Opération passée")


Opération passée
